In [ ]:
import random 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# coin toss
random.choice(['H', 'T'])

In [ ]:
# die roll
random.choice([1, 2, 3, 4, 5, 6]) 

In [ ]:
# bus is late anywhere from 0 to 10 minutes, uniform distribution
random.uniform(0, 10) 

# how is this different from random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])?

# Sampling from probability distribution

What happens when you change the number of experiments (i.e. coin flip, die roll, bus arrival time measurement) from 20 to 100? 1000? 5?

In [ ]:
def sample_coin(k): 
    flips = random.choices(['H', 'T'], k=k) # This is sampling from uniform distribution
    return flips

In [ ]:
flips = sample_coin(20)
sns.histplot(flips, stat='density') # This is empirical distribution of simulated data

In [ ]:
def sample_dice(k):
    rolls = random.choices([1, 2, 3, 4, 5, 6], k=k)
    return rolls

In [ ]:
rolls = sample_dice(20)
sns.histplot(rolls, stat='density', bins=range(1,7))

In [ ]:
def sample_bus(n):
    times = []
    for bus in range(n):
        t = random.uniform(0, 10)
        times.append(t)
    return times

In [ ]:
times = sample_bus(20)
sns.histplot(times, stat='density', bins=range(0,11))

# Law of large numbers

Sample mean approaches population mean

In [ ]:
# expected value is 3.5 (lec 14)

rolls = sample_dice(20)
sum(rolls)/len(rolls) # calculate sample mean

In [ ]:
ks = range(1, 1001) 
# Number of samples drawn from uniform distribution, try 1 to 1000

all_means = []

for k in ks:
    rolls = sample_dice(k)
    m = sum(rolls)/len(rolls) # calculate sample mean
    all_means.append(m) # save sample mean

fig, ax = plt.subplots(1)
sns.scatterplot(x=ks, y=all_means, label='sample mean')
ax.set_xlabel('Number of samples')
ax.set_ylabel('Mean of observed data')
ax.axhline(3.5, color='k', label='expected value')
ax.legend()

In [ ]:
ks = range(1, 1001) 
# Number of samples drawn from uniform distribution, try 1 to 1000

all_means = []

for k in ks:
    times = sample_bus(k)
    m = sum(times)/len(times) # calculate sample mean
    all_means.append(m) # save sample mean

fig, ax = plt.subplots(1)
sns.scatterplot(x=ks, y=all_means, label='sample mean')
ax.set_xlabel('Number of samples')
ax.set_ylabel('Mean of observed data')
ax.axhline(5, color='k', label='expected value')
ax.legend()

# Central Limit Theorem

Distribution of sample means approaches normal distribution.

Try the different values for `num_rolls_per_expt` and `num_buses_per_expt`. Do you notice anything change in the mean and std of sample means? In the shape?

In [ ]:
num_experiment = 1000 # number of times you repeat the experiment

num_rolls_per_expt = 5 # number of samples you draw from uniform distribution
#num_rolls_per_expt = 20
#num_rolls_per_expt = 80
#num_rolls_per_expt = 320

all_means = []


for roll in range(num_experiment): 
    rolls = sample_dice(num_rolls_per_expt) 
    m = sum(rolls)/len(rolls) # calulate sample mean
    all_means.append(m)

fig, ax = plt.subplots(1)
sns.histplot(all_means, label='sample means', stat='density')
ax.set_xlabel('Sample mean of die roll')
ax.axvline(3.5, label='expected value', color='k', linestyle='-.')
ax.legend()

print('num_experiment:', num_experiment)
print('num_rolls_per_expt:', num_rolls_per_expt)
print('mean of sample means:', pd.DataFrame(all_means).mean()[0])
print('standard deviation of sample means:', pd.DataFrame(all_means).std()[0])

In [ ]:
num_experiment = 1000  # number of times you repeat the experiment

num_buses_per_expt = 5 # number of samples you draw from uniform distribution
#num_buses_per_expt = 20
#num_buses_per_expt = 80
#num_buses_per_expt = 320

all_means = []

for measurement in range(num_experiment):
    times = sample_bus(num_buses_per_expt) 
    m = sum(times)/len(times) # calulate sample mean
    all_means.append(m)

fig, ax = plt.subplots(1)
sns.histplot(all_means, label='sample means', stat='density')
ax.set_xlabel('Sample mean of bus arrival times')
ax.axvline(5, label='expected value', color='k', linestyle='-.')
ax.legend()

print('num_experiment:', num_experiment)
print('num_rolls_per_expt:', num_buses_per_expt)
print('mean of sample means:', pd.DataFrame(all_means).mean()[0])
print('standard deviation of sample means:', pd.DataFrame(all_means).std()[0])

# Hypothesis test

(a)

Null hypothesis: The coin is fair.

Alternative: No, it’s biased towards heads.

(b)

Null hypothesis: The coin is fair.

Alternative: No, it's not.

In [ ]:
actual_coin = random.choices(['H', 'T'], k=400, weights=[0.4, 0.6])
pd.DataFrame(actual_coin, columns=['coin']).to_csv('coins_400.csv', index=False)

In [ ]:
coins_400 = pd.read_csv('coins_400.csv')
sns.histplot(coins_400, x='coin', stat='density')
coins_400.head()

In [ ]:
# (a) percent of heads
heads = (coins_400['coin']=='H')
t1 = sum(heads) / len(coins_400)
t1

In [ ]:
# (b) | percent of heads - 50% | 
t2 = abs(t1 - 0.5)
t2

In [ ]:
# 1. Make a lot of simulated data under the null hypothesis
# 2. Calculate the statistics t1 and t2 from simulated data

statistic = pd.DataFrame(columns=['t1', 't2'])

for simulation in range(1000):
    simulated_coins = pd.DataFrame(sample_coin(400), columns=['coin'])
    heads = (simulated_coins['coin']=='H') # True, False array
    sim_t1 = sum(heads) / len(simulated_coins) # proportion of heads
    sim_t2 = abs(sim_t1-0.5) # difference from 50%
    statistic.loc[len(statistic)] = [sim_t1, sim_t2]
statistic

In [ ]:
# 3. Plot empirical distribution of t1 and t2 (under the null)

fig, ax = plt.subplots(1)
sns.histplot(data=statistic, x='t1', label='Under the null')
ax.axvline(t1, color='k', linestyle='-.', label='In dataset')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1)
sns.histplot(data=statistic, x='t2', label='Under the null')
ax.axvline(t2, color='k', linestyle='-.', label='In dataset')
ax.legend()

# Draft

In [ ]:
draft = pd.read_csv('~/F24-public/data/draft70yr.csv')

In [ ]:
month_data = draft.groupby('Month').mean()
month_data

In [ ]:
sns.barplot(data=month_data, x='Month', y='Pick')